In [1]:


%pip install -qU langchain duckduckgo-search langchain-community



Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import getpass
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool


if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq:")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama3-8b-8192", model_provider="groq")

def is_simple(input_text: str) -> bool:
    prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        Given the following input: "{text}", determine whether it is simple.
        Reply with 'yes' for simple and 'no' for complex.
        """
    )  
    chain = LLMChain(llm=llm, prompt=prompt)
    response = chain.run(text=input_text).strip().lower()
    return response == "yes"


def split_into_simple_queries(input_text: str) -> list:
    """Splits complex input into multiple simple queries for a search engine tool."""
    prompt = PromptTemplate(
        input_variables=["text"],
        template="""
        Break down the following complex question into multiple simpler search queries, break them down so that if the task is complex we summarize it, do not add extra text:
        "{text}"
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(text=input_text).strip().split('\n')

@tool
def process_query(input_text: str):
    """Decides if the query is simple or complex and processes accordingly for a search engine tool."""
    if is_simple(input_text):
        return {"is_simple": True,"queries": [input_text]}
    else:
        return {"is_simple": False ,"queries": split_into_simple_queries(input_text)}

# Example Usage
if __name__ == "__main__":
    user_input = "Do the dishes"
    result = process_query(user_input)
    print("Final Output:", result)


ImportError: Unable to import langchain_groq. Please install with `pip install -U langchain-groq`

NameError: name 'SystemMessage' is not defined

In [12]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [ ]:

import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that helps breaks down tasks. Help summarize these tasks based on complexity, if the task is simple then say BASIC, if complex break down step by step on how to achieve the task",
    ),
    ("human", "i need to wash the dishes"),
    ]
ai_msg = llm.invoke(messages)
ai_msg

print(ai_msg.content)



A simple task. I would classify washing the dishes as a BASIC task. Here's a brief summary:

1. Gather dish soap and warm water.
2. Fill the sink with warm water and add dish soap.
3. Submerge the dirty dishes in the soapy water.
4. Scrub the dishes with a sponge or dish brush.
5. Rinse the dishes thoroughly with clean water.
6. Dry the dishes with a towel.

That's it! A quick and easy task to get your dishes clean.
